In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/digit-recognizer/sample_submission.csv
/kaggle/input/digit-recognizer/train.csv
/kaggle/input/digit-recognizer/test.csv


In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib as plt
import seaborn as sns

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
(X_train, y_train), (X_test, y_test) = keras.datasets.mnist.load_data()

11490434/11490434 [==============================] - 0s 0us/step


In [4]:
X_train = X_train.astype('float32') / 255
X_test = X_test.astype('float32') / 255

X_train = np.expand_dims(X_train, -1)
X_test = np.expand_dims(X_test, -1)

In [5]:
print('X_train shape:', X_train.shape)
print('y_train shape:', y_train.shape)
print('train samples:', X_train.shape[0])
print('test samples:', X_test.shape[0])

X_train shape: (60000, 28, 28, 1)
y_train shape: (60000,)
train samples: 60000
test samples: 10000


In [6]:
num_classes = 10
input_shape = (28, 28, 1)

model = keras.Sequential([
    keras.layers.Input(shape=input_shape),
    keras.layers.Conv2D(64, kernel_size=(3, 3), activation='relu'),
    keras.layers.Conv2D(64, kernel_size=(3, 3), activation='relu'),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),
    keras.layers.Conv2D(128, kernel_size=(3, 3), activation='relu'),
    keras.layers.Conv2D(128, kernel_size=(3, 3), activation='relu'),
    keras.layers.GlobalAveragePooling2D(),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(num_classes, activation='softmax')
])

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 64)        640       
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 64)        36928     
                                                                 
 max_pooling2d (MaxPooling2  (None, 12, 12, 64)        0         
 D)                                                              
                                                                 
 conv2d_2 (Conv2D)           (None, 10, 10, 128)       73856     
                                                                 
 conv2d_3 (Conv2D)           (None, 8, 8, 128)         147584    
                                                                 
 global_average_pooling2d (  (None, 128)               0         
 GlobalAveragePooling2D)                                

In [8]:
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(),
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    metrics=[
        keras.metrics.SparseCategoricalAccuracy(name="acc"),
    ],
)

In [9]:
batch_size=128
epochs = 20

callbacks = [
    keras.callbacks.ModelCheckpoint(filepath="model_at_epoch_{epoch}.keras"),
    keras.callbacks.EarlyStopping(monitor="val_loss", patience=2)
]

model.fit(
    X_train,
    y_train,
    batch_size=batch_size,
epochs=epochs,
validation_split=0.15,
callbacks=callbacks)

Epoch 1/20
399/399 [==============================] - 10s 13ms/step - loss: 0.7418 - acc: 0.7482 - val_loss: 0.1413 - val_acc: 0.9593
Epoch 2/20
399/399 [==============================] - 5s 11ms/step - loss: 0.2094 - acc: 0.9375 - val_loss: 0.0719 - val_acc: 0.9791
Epoch 3/20
399/399 [==============================] - 5s 11ms/step - loss: 0.1430 - acc: 0.9583 - val_loss: 0.0585 - val_acc: 0.9824
Epoch 4/20
399/399 [==============================] - 5s 11ms/step - loss: 0.1176 - acc: 0.9656 - val_loss: 0.0524 - val_acc: 0.9840
Epoch 5/20
399/399 [==============================] - 5s 11ms/step - loss: 0.0985 - acc: 0.9715 - val_loss: 0.0471 - val_acc: 0.9859
Epoch 6/20
399/399 [==============================] - 5s 11ms/step - loss: 0.0839 - acc: 0.9751 - val_loss: 0.0364 - val_acc: 0.9906
Epoch 7/20
399/399 [==============================] - 5s 12ms/step - loss: 0.0767 - acc: 0.9780 - val_loss: 0.0342 - val_acc: 0.9896
Epoch 8/20
399/399 [==============================] - 5s 11ms/step -

In [10]:
score = model.evaluate(X_test, y_test, verbose=0)

In [11]:
prediction = model.predict(X_test)

313/313 [==============================] - 1s 2ms/step


In [12]:
prediction[0]

array([1.2513242e-09, 2.4204944e-08, 4.9889150e-06, 2.4020288e-08,
       3.6691716e-09, 5.9943978e-10, 7.8984042e-17, 9.9999225e-01,
       9.6838378e-11, 2.6823393e-06], dtype=float32)

In [13]:
y_test

array([7, 2, 1, ..., 4, 5, 6], dtype=uint8)

In [14]:
tf.saved_model.save(model, "/tmp/cnn/1/")